In [1]:
import sys
sys.path.append('../')
import joblib
from helper_files.etv_model import ETVModel
import joblib
import pandas as pd
from helper_files.plotting_functions import get_opponent_df, generate_fv_grid, plot_heatmap, generate_cp_grid
from tqdm import tqdm
tqdm.pandas()
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
import statsmodels.api as sm


/Users/bradeneberhard/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
models = joblib.load('../data/models/xgb_models.jblb')
data_path = '../data/processed/data_1003.jblb'
DATA = joblib.load(data_path)['df']
DATA['etv'] = models['etv_model'].predict(DATA[models['cp_features']])

In [3]:
def get_opponent_data(df, features):
    opponent_data = df[features].copy()
    opponent_data['thrower_x'] = opponent_data.receiver_x * -1
    opponent_data.loc[:, 'thrower_y'] = (120 - opponent_data.loc[:, 'receiver_y']).clip(lower=20, upper=100)
    opponent_data.loc[:, 'possession_num'] += 1
    if 'possession_num' in opponent_data.columns:
        opponent_data.loc[:, 'possession_num'] += 1

    if 'possession_throw' in opponent_data.columns:
        opponent_data.loc[:, 'possession_throw'] = 1

    if 'score_diff' in opponent_data.columns:
        opponent_data.loc[:, 'score_diff'] = -opponent_data.loc[:, 'score_diff']
    opponent_data = opponent_data.drop(['receiver_x', 'receiver_y'], axis=1)
    return opponent_data

fv_scaler = models['etv_model'].fv_scaler
thrower_fv_features = models['fv_model']['features']
receiver_fv_features = [x.replace('thrower', 'receiver') for x in models['fv_model']['features']]
DATA['thrower_fv'] = models['fv_model']['model'].predict_proba(fv_scaler.transform(DATA[thrower_fv_features]))[:,1]
DATA['receiver_fv'] = models['fv_model']['model'].predict_proba(fv_scaler.transform(DATA[receiver_fv_features].values))[:,1]
DATA.loc[DATA.receiver_y > 100, 'receiver_fv'] = 1
opponent_data = get_opponent_data(DATA, receiver_fv_features)
DATA['opponent_fv'] = models['fv_model']['model'].predict_proba(fv_scaler.transform(opponent_data.values))[:,1]
DATA['ec'] = DATA['opponent_fv'] - DATA['thrower_fv']  
DATA.loc[DATA['completion'] == 1, 'ec'] = DATA['receiver_fv'] - DATA['thrower_fv']


/Users/bradeneberhard/miniforge3/lib/python3.9/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/bradeneberhard/miniforge3/lib/python3.9/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [27]:
DATA

,thrower,thrower_x,thrower_y,receiver,receiver_x,receiver_y,turnover,possession_num,possession_throw,game_quarter,...,completion,gameDate,year,hockey_assist,thrower_completion_percentage,etv,thrower_fv,receiver_fv,opponent_fv,ec
213828,aseibert,15.80,45.15,NaN,1.74,48.95,1,1,3,3,...,0,2021-06-19,2021,0,0.0,0.681972,0.618881,0.717434,0.439250,-0.179631
37999,mgoetsch,0.93,58.45,NaN,1.70,53.93,1,1,6,1,...,0,2023-07-21,2023,0,0.0,0.747723,0.776097,0.768900,0.299133,-0.476965
268706,ejoyce,17.79,93.90,NaN,15.47,109.54,1,1,7,1,...,0,2022-06-11,2022,0,0.0,0.862213,0.855622,1.000000,0.156998,-0.698624
243794,dbaker1,1.67,47.76,alopezesc,1.67,47.76,1,2,3,2,...,0,2022-05-21,2022,0,0.0,0.588173,0.709549,0.709549,0.410703,-0.298846
221402,rjones,-23.06,52.96,bobryan,-24.41,57.09,1,1,3,4,...,0,2021-08-29,2021,0,0.0,0.661690,0.781347,0.775387,0.095595,-0.685753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76798,bthomsen,23.24,46.52,dferriter,16.24,49.15,0,1,1,4,...,1,2023-07-25,2023,0,100.0,0.716369,0.700893,0.756066,0.030487,0.055173
76809,cnelson,-9.33,90.07,wanderson,-4.89,97.35,0,1,2,4,...,1,2023-07-25,2023,1,100.0,0.845692,0.873888,0.921795,0.463760,0.047908
76824,msutton,-12.10,66.62,wgoal,-16.76,76.74,0,1,1,4,...,1,2023-07-25,2023,0,100.0,0.727698,0.742809,0.772306,0.034075,0.029497
82751,ctexeira,-16.80,37.29,edavis,-22.35,32.38,0,1,4,4,...,1,2024-07-21,2024,0,100.0,0.620858,0.685771,0.633368,0.044911,-0.052404


In [19]:
throw_counts = DATA.groupby('thrower').size()
valid_throwers = throw_counts[throw_counts > 20].index
filtered_DATA = DATA[DATA['thrower'].isin(valid_throwers)]


In [26]:
filtered_DATA.groupby('thrower').ec.sum().sort_values()

thrower
bobryan     -19.144159
bwalsh      -18.161018
ebilolo     -16.965416
jhustad     -14.956038
jfelton     -14.884804
               ...    
srueschem    21.935078
echartock    22.243586
aroy         22.361366
ataylor      23.359699
pjanas       30.082924
Name: ec, Length: 1144, dtype: float32

In [28]:
filtered_DATA.groupby('receiver').ec.sum().sort_values()

receiver
apillsbur    -2.007469
nlongo       -1.995424
hmcnamara    -1.749836
ilee         -1.605375
kcarroll     -1.338622
               ...    
smcdougal    47.007339
eswiatek     47.343388
asjogren     54.361393
jfairfax     54.471550
bjagt        61.867496
Name: ec, Length: 1561, dtype: float32